## Train BERT model



In [1]:
import os
from glob import glob
import pandas as pd

def get_lang(file):
    return os.path.splitext(os.path.basename(file))[0]

def load_df(file):
    dialect = get_lang(file)
    
    df = pd.read_table(file, names=["id", "text", "polarity"], index_col=0)
    #df["dialect"] = dialect
    df["label"] = 1
    df.loc[df["polarity"] == 'P', "label"] = 2
    df.loc[df["polarity"] == 'N', "label"] = 0
    return df

train_files = glob("../data/tass2020/train/*.tsv")
dev_files = glob("../data/tass2020/dev/*.tsv")
test_files = glob("../data/tass2020/test1.1/*.tsv")

train_dfs = {get_lang(file):load_df(file) for file in train_files}
dev_dfs = {get_lang(file):load_df(file) for file in dev_files}
test_dfs = {get_lang(file):load_df(file) for file in test_files}

train_df = pd.concat(train_dfs.values())
dev_df = pd.concat(dev_dfs.values())
test_df = pd.concat(test_dfs.values())

print(len(train_df), len(dev_df), len(test_df))

train_df.columns, dev_df.columns, test_df.columns

4802 2443 7264


/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar i

(Index(['text', 'polarity', 'label'], dtype='object'),
 Index(['text', 'polarity', 'label'], dtype='object'),
 Index(['text', 'polarity', 'label'], dtype='object'))

## Preprocessing

Convertimos todos los usuarios al string "usuario"



In [2]:
import re

regex = re.compile(r"@[a-zA-Z0-9_]{0,15}")


regex.sub("usuario", "@infobae @OnlyInPeronia jajaja me muero")


'usuario usuario jajaja me muero'

In [3]:
user_regex = re.compile(r"@[a-zA-Z0-9_]{0,15}")
url_regex = re.compile(
    "((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|co|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"
)
def preprocess_tweet(text):
    """
    Basic preprocessing
    """
    text = user_regex.sub("usuario", text)
    text = url_regex.sub("url", text)
    
    return text

train_df["text"] = train_df["text"].apply(preprocess_tweet)
dev_df["text"] = dev_df["text"].apply(preprocess_tweet)
test_df["text"] = test_df["text"].apply(preprocess_tweet)


In [4]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

model_name = 'dccuchile/bert-base-spanish-wwm-cased'

device = "cuda" if torch.cuda.is_available() else "cpu"

model = BertForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=3)

model.config.hidden_dropout_prob = 0.20
model = model.to(device)
model.train();



tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased a

Veamos primero las longitudes (a ver si no hay nada mal cargado)

In [5]:
from datasets import Dataset, Value, ClassLabel, Features

examples = pd.concat([train_df, dev_df])

features = Features({
    'text': Value('string'),
    'label': ClassLabel(num_classes=3, names=["neg", "neu", "pos"])
})

train_dataset = Dataset.from_pandas(train_df[["text", "label"]], features=features)
dev_dataset = Dataset.from_pandas(dev_df[["text", "label"]], features=features)
test_dataset = Dataset.from_pandas(test_df[["text", "label"]], features=features)

PyTorch version 1.4.0 available.


In [6]:
train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=3, names=['neg', 'neu', 'pos'], names_file=None, id=None)}

In [7]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

batch_size = 32

eval_batch_size = 16

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)


Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 4802 examples in 15287950 bytes .
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 2443 examples in 7788684 bytes .
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 7264 examples in 23088522 bytes .


In [8]:
def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['label']})
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 4802 examples in 15324030 bytes .
Set __getitem__(key) output type to torch for ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 2443 examples in 7805828 bytes .
Set __getitem__(key) output type to torch for ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.
Testing the mapped function outputs
Testing finished, running the mapping function on the dataset


Done writing 7264 examples in 23139498 bytes .
Set __getitem__(key) output type to torch for ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] columns  (when key is int or slice) and don't output other (un-formatted) columns.


In [9]:
from torchtext import data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_it = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
dev_it = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size)
test_it = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

device

device(type='cuda')

In [10]:



def compute_metrics(labels, preds):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


def evaluate(model, it):
    """
    Calculates labels and predictions for it
    
    Returns
    -------
    
    (labels, preds): torch.Tensor
    
    where labels are the true labels
    and preds are the predictions
    
    """
    model.eval()
    preds = []
    true = []
    losses = []
    with torch.no_grad():
        for i, batch in enumerate(tqdm(it)):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            losses.append(loss.item())
            outs = torch.softmax(outputs.logits, dim=1)
            
            true.append(batch['labels'].cpu())
            preds.append(outs.cpu())
    return np.array(losses).mean(), torch.cat(true), torch.cat(preds).argmax(1)


In [11]:
from tqdm.auto import tqdm
from torch import nn
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AdamW, get_linear_schedule_with_warmup


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.train().to(device)

epochs = 10

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)


num_training_steps = epochs * len(train_it)
num_warmup_steps = num_training_steps // 10
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps
)

for epoch in range(epochs):
    train_losses = []
    for i, batch in enumerate(tqdm(train_it)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    
    train_loss = np.array(train_losses).mean()
    dev_loss, dev_labels, dev_preds = evaluate(model, dev_it)
    dev_metrics = compute_metrics(dev_labels, dev_preds) 
    
    print(f"Epoch {epoch:<2}")
    print(f"Train loss {train_loss:.4f}")
    print(f"Dev loss {dev_loss:.4f}")
    print(f"Dev metrics {dev_metrics}")


Epoch 0 
Train loss 1.0225
Dev loss 0.8845
Dev metrics {'accuracy': 0.5800245599672533, 'f1': 0.5273043035005734, 'precision': 0.6051953379716025, 'recall': 0.5576677030266969}



Epoch 1 
Train loss 0.7186
Dev loss 0.7838
Dev metrics {'accuracy': 0.6557511256651658, 'f1': 0.6487443410843955, 'precision': 0.6512699498404012, 'recall': 0.6498570983064563}



Epoch 2 
Train loss 0.4615
Dev loss 0.8428
Dev metrics {'accuracy': 0.6672124437167417, 'f1': 0.666598901550708, 'precision': 0.6666833824123274, 'recall': 0.6668643310869594}



Epoch 3 
Train loss 0.2727
Dev loss 0.9440
Dev metrics {'accuracy': 0.6598444535407286, 'f1': 0.6599412534946892, 'precision': 0.6627497177983619, 'recall': 0.6581376777968915}



Epoch 4 
Train loss 0.1623
Dev loss 1.1059
Dev metrics {'accuracy': 0.6557511256651658, 'f1': 0.6474144558141375, 'precision': 0.654672224443276, 'recall': 0.6465583383009803}



Epoch 5 
Train loss 0.0993
Dev loss 1.2006
Dev metrics {'accuracy': 0.6635284486287352, 'f1': 0.6558691210795763, 'precision': 0.6538927217178321, 'recall': 0.6620069116834751}



Epoch 6 
Train loss 0.0756
Dev loss 1.2851
Dev metrics {'accuracy': 0.6565697912402784, 'f1': 0.6506782375001033, 'precision': 0.6505409574102564, 'recall': 0.6613688474139242}



Epoch 7 
Train loss 0.0538
Dev loss 1.2985
Dev metrics {'accuracy': 0.6573884568153909, 'f1': 0.6609376840923649, 'precision': 0.6754349423222621, 'recall': 0.6560415107195824}



Epoch 8 
Train loss 0.0326
Dev loss 1.4343
Dev metrics {'accuracy': 0.6299631600491199, 'f1': 0.6387203807640974, 'precision': 0.6770675321978743, 'recall': 0.6374226806409898}



Epoch 9 
Train loss 0.0272
Dev loss 1.3530
Dev metrics {'accuracy': 0.6643471142038477, 'f1': 0.6649911260793954, 'precision': 0.6719269024188338, 'recall': 0.6612119098544086}


In [13]:
path = "../models/beto-sentiment-analysis"
model.save_pretrained(path)
tokenizer.save_pretrained(path)

('../models/beto-sentiment-analysis/vocab.txt',
 '../models/beto-sentiment-analysis/special_tokens_map.json',
 '../models/beto-sentiment-analysis/added_tokens.json')